# Metaprogramming and fundamental concepts of plugin infrastructures

Metaprogramming allows computer programs to treat other programs like data. This allows metaprograms to read other programs and themselves during runtime, optimize them or generate new functionalities.

One example of metaprogramming are plugin infrastructures.

### Metaclasses in Python

In Python exists two kind of classes:
- New Class: Inherits from object. It is the only way to create a class in Python 3.
- Classic Class: They were retained through Python 2.7 for backwards compatibility, and were removed in Python 3.

When executing the following code in Python 2.7:
```python
import sys

print(sys.version)

class BaseClass:
  pass

x = BaseClass()
print(x.__class__)
print(type(x))

```
The output is:
```
2.7.18 (default, Nov 21 2022, 21:13:16) [GCC 12.2.0]
__main__.BaseClass
<type 'instance'>
```
In a classic class an instance of an old-style class is always implemented from a single built-in type called *instance*.

Here is the same output in Python 3.10:
```
3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
<class '__main__.BaseClass'>
<class '__main__.BaseClass'>
```

In Python everything is an object also a class. Therefore a class is also an object with a type:

In [1]:
import sys

print(sys.version)

class BaseClass:
  pass

x = BaseClass()
print(x.__class__)
print(type(x))
print('Type of the class is: ', type(BaseClass))   # type of the class is type  (metaclass)

3.12.0 (tags/v3.12.0:0fb18b0, Oct  2 2023, 13:03:39) [MSC v.1935 64 bit (AMD64)]
<class '__main__.BaseClass'>
<class '__main__.BaseClass'>
Type of the class is:  <class 'type'>


### Type of built in base classes
Also the built in base classes are of the type *type*:

In [2]:
for i in int, float, dict, list, tuple:
  print(type(i))   # type of the class is type  (metaclass) (all classes are instances of type)

<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>


The *type* of the class type is also *type*

In [3]:
print(type(type))  # type of the class is type  (metaclass  (type is instance of itself))   (type is metaclass)

<class 'type'>


### Creating instances of the type class
Instances of the *type* class can be created by calling ```type(<name>, <bases>, <dct>)``` with three arguments:
- ```<name>```: Name of the dynamicaly created class. Specifies the *__name__* attribut
- ```<bases>```: Tuple of the base classes from which the class inherits. Specifies the *__base__* attribute.
- ```<dct>```: Specifies a namespace dictionary containing definitions for the class body. Becomes the *__dict__* attribute.

In [11]:
def func(obj):
    print('attr =', obj.attr_value)

baseClass = type('BaseClass',(),{'attr_value': 100, 'attr_func': func})  # type(name, bases, dict)  (metaclass)
x = baseClass()     
print(x.attr_value)
x.attr_func()

100
attr = 100


### Creation of new instances
The expression ```BaseClass()``` creates a new instance of the *baseClass*.
When calling ```BaseClass()``` first the method ```__call__()``` of the parent class is called.
The the methodes ```__new__()``` and ```__init__()``` from the ***BaseClass*** gets called.

In [12]:
def new(cls):
    x = object.__new__(cls) # object.__new__(cls)  (metaclass)
    x.attr = 100
    return x

BaseClass.__new__ = new

baseClass = BaseClass()
baseClass.attr

100

Metaclasses can be used as a template for the creation of classes. Tis is called class factory.
Because it is not possible to add a new ```__new__()``` method to the *type* class we need a metaclass as following:

In [13]:
class Meta(type):
    def __new__(cls, name, bases, dct):
        x = super().__new__(cls, name, bases, dct)
        x.attr = 100
        return x

Now we cann use our metaclass *Meta* as class factory (template) for the creation of new classes:

In [15]:
class BaseClass(metaclass=Meta):
    pass

print(BaseClass.attr)

100


The same can also be done using a class decorator:

In [16]:
def decorator(cls):
    class NewClass(cls):
        attr = 100
    return NewClass

@decorator
class BaseClass:
    pass

print(BaseClass.attr)

100


### Creation of pluging using metaclasses:
The concept of metaclasses can be used to create plugins.
A plugin application has the following phases to interact with plugins:
- ***Discovery:*** Mechanism by which a running application can find out which plugins it has at its disposal. 
- ***Registration:***: Mechanism by which a plugin registers itself in the application.
- ***Application Hooks:*** Places where the plugin can *attach* itself to the application.
- ***Exposing application API to plugins:***: the application gives the plugin access to itself.

In [17]:
class IPluginRegistry(type):
    plugins = []

    def __init__(cls, name, bases, attrs):
        if name != 'IPlugin':
            IPluginRegistry.plugins.append(cls)

class IPlugin(object, metaclass=IPluginRegistry):
    pass

class PluginA(IPlugin):
    pass

class PluginB(IPlugin):
    pass

print(IPluginRegistry.plugins)

[<class '__main__.PluginA'>, <class '__main__.PluginB'>]


### Example in the project

Example plugin:
```python
from models.nodes.subnet_nodes.subnet_node_layers.firewall_layer import IFirewallPlugin
from typing import TYPE_CHECKING
if TYPE_CHECKING:
    from models import NetworkPackage


class Plugin(IFirewallPlugin):
    """
    This class defines the default method that must be implemented in all firewall plugins.

    Methods
    ----------
    filter_request(message)
        Returns if the given message should be filtered or not.
    """

    def filter_request(self, network_package: 'NetworkPackage') -> tuple[bool, str, str]:
        """
        Returns if the given network package should be filtered or not.

        Parameters
        ----------
        network_package: NetworkPackage
            The network package that should be filtered or not.

        Returns
        ----------
        tuple[bool, str, str]
            If the message should be filtered, the reason for it and the source.
        """
        
        return (False, "", "Default Filter Plugin")
```

Register the plugins:

```python
if len(os.listdir('./models/nodes/subnet_nodes/subnet_node_layers/firewall_layer/plugins')) == 0:
    sys.exit("No filter plugin detcted. Please place default filter plugin in the filter plugis directory.")
sys.path.append('./models/nodes/subnet_nodes/subnet_node_layers/firewall_layer/plugins')
self.plugins = [
    importlib.import_module(f.split('.')[0], '.').Plugin()
    for f in next(os.walk('models/nodes/subnet_nodes/subnet_node_layers/firewall_layer/plugins'))[2]
]
```
Application hook:
```python
for plugin in self.plugins:
    filter_response = plugin.filter_request(network_package)
    if filter_response[0]:
        alert = Alert(msg=filter_response[1], source="Filter Plugin: "+str(filter_response[2], network_package=network_package))
        self.notify(alert)
        return None
```